# Competition Overview

- Title : [image-matching-challenge-2025][1]
- Challenge goal: Extend last year’s Image Matching Challenge by not only reconstructing 3D scenes from images but also automatically grouping relevant images and discarding unrelated ones.

- Problem addressed: Real-world photo collections often mix in unrelated or visually similar but incorrect images (e.g. two sides of a monument or a coffee cup shot near a landmark), which confuses Structure-from-Motion (SfM) pipelines.

- Current limitations: SfM methods perform well in controlled settings with GPS or video data, but struggle on large, messy, crowdsourced image sets where such metadata may be missing or unreliable.

- Impact: By improving automatic grouping and filtering of images, the competition aims to make SfM more robust for large-scale 3D reconstruction tasks—benefiting fields like urban planning and scientific research.


# Realted Work
- SFM
- Image Matching

















[1]: https://www.kaggle.com/competitions/image-matching-challenge-2025

## 1. Libary import & Basic Setup

In [21]:
import pandas as pd
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [30]:
parent_dir = os.path.abspath(os.path.join(cwd, os.pardir))
data_path = os.path.join(parent_dir, 'data')

In [32]:
train_root_path = os.path.join(parent_dir, 'data', 'train')
test_root_path = os.path.join(parent_dir, 'data', 'test')

In [39]:
train_labels = pd.read_csv(os.path.join(data_path, 'train_labels.csv'))
train_threshold = pd.read_csv(os.path.join(data_path, 'train_thresholds.csv'))
sample_submission = pd.read_csv(os.path.join(data_path, 'sample_submission.csv'))

In [40]:
train_labels.head()

,dataset,scene,image,rotation_matrix,translation_vector
0,imc2023_haiper,fountain,fountain_image_116.png,0.122655949;0.947713775;-0.294608417;0.1226706...,0.093771314;-0.803560988;2.062001533
1,imc2023_haiper,fountain,fountain_image_108.png,0.474305910;0.359108654;-0.803787832;0.2888416...,0.358946647;-0.797557548;1.910906929
2,imc2023_haiper,fountain,fountain_image_101.png,0.565115476;-0.138485064;-0.813305838;0.506678...,0.146922468;-0.981392596;2.009002852
3,imc2023_haiper,fountain,fountain_image_082.png,-0.308320392;-0.794654112;0.522937261;0.948141...,0.206413831;-1.174321103;3.667167680
4,imc2023_haiper,fountain,fountain_image_071.png,-0.569002830;-0.103808175;0.815757098;0.778745...,-0.015140892;-1.334052012;3.488936597


In [42]:
train_threshold.head()

,dataset,scene,thresholds
0,amy_gardens,peach,0.01;0.02;0.05;0.1;0.25;0.5
1,ETs,ET,0.002;0.005;0.008;0.01;0.02;0.04
2,ETs,another_ET,0.002;0.005;0.008;0.01;0.02;0.04
3,fbk_vineyard,vineyard_split_1,0.01;0.02;0.05;0.1;0.25;0.5
4,fbk_vineyard,vineyard_split_2,0.01;0.02;0.05;0.1;0.25;0.5


In [43]:
sample_submission.head()

,image_id,dataset,scene,image,rotation_matrix,translation_vector
0,ETs_another_et_another_et001.png_public,ETs,cluster0,another_et_another_et001.png,0.984538905;0.811159853;0.681088421;0.84509285...,0.427962849;0.408638396;0.184856919
1,ETs_another_et_another_et002.png_public,ETs,cluster0,another_et_another_et002.png,0.557144821;0.589081187;0.788143210;0.10120414...,0.112591455;0.280966383;0.338320921
2,ETs_another_et_another_et003.png_public,ETs,cluster0,another_et_another_et003.png,0.070922149;0.331333475;0.055966106;0.76511642...,0.729445911;0.458064343;0.035260485
3,ETs_another_et_another_et004.png_public,ETs,cluster0,another_et_another_et004.png,0.577042073;0.978031237;0.157452522;0.69871702...,0.728382837;0.491174888;0.826992214
4,ETs_another_et_another_et005.png_public,ETs,cluster0,another_et_another_et005.png,0.684873837;0.658222912;0.810479111;0.15049752...,0.687218658;0.036219706;0.266501550


## 2. Data Description

- **train_labels.csv**  
  - 데이터셋(train)에 포함된 이미지 목록과 정답 정보  
  - `dataset`: 데이터셋 고유 ID  
  - `scene`: 장면 고유 ID  
  - `image`: 이미지 파일명  
  - `rotation_matrix`: 첫 번째 예측 대상. 3×3 행렬을 행 우선(row-major) 방식으로 펼쳐 놓고 “;”로 구분  
  - `translation_vector`: 두 번째 예측 대상. 3차원 벡터, 값들은 “;”로 구분  

- **train_thresholds.csv**  
  - 평가 시 사용할 거리 임계치 목록  
  - `dataset`: 데이터셋 고유 ID  
  - `scene`: 장면 고유 ID  
  - `thresholds`: 평가용 거리 임계치(단위: 미터), “;”로 구분  

- **[train/test]/[dataset]/*.png**  
  - 한 개 이상의 위치에서 촬영된 이미지 집합  
  - 예: `train/imc2024_lizard_pond` 폴더에는 ‘lizard’와 ‘pond’라는 두 장면의 이미지가 섞여 있음  
  - 단일 이미지(“outliers”)만 들어 있는 폴더도 있을 수 있음  
  - 참고: `imc2023_haiper`에는 의도치 않게 서로 공간적으로 겹치는 두 장면(‘bike_image_*’와 ‘chairs_image_*’)이 포함되어 있음. 이를 까다로운 사례로 다루거나 무시할 수 있음.  

- **[train/test]/[dataset]/LICENSE.txt**  
  - 각 데이터셋의 라이선스 정보  

> 참고: 공개된 test 폴더에는 학습 세트의 일부 장면만 포함되어 있으며, 예시 용도로 제공됩니다. 실제 채점 시에는 다른 데이터로 대체됩니다.

- **sample_submission.csv**  
  - 무작위로 생성된 예시 제출 파일  
  - `image_id`: 각 행의 고유 ID (제출 파일에 반드시 포함)  
  - `dataset`: 데이터셋 고유 ID  
  - `scene`: 장면 고유 ID (제출 시에는 이미지 그룹화를 나타내는 임의 레이블로 사용)  
  - `image`: 이미지 파일명  
  - `rotation_matrix`: 회전 행렬 (위와 동일)  
  - `translation_vector`: 이동 벡터 (위와 동일)  

'/Users/iyongjeong/WORK/kaggle/matching/notebooks'